# Tutorial: StructureMap

## Import libraries

In [1]:
# Import structuremap functions
import structuremap.utils
structuremap.utils.set_logger()
from structuremap.processing import download_alphafold_cif, download_alphafold_pae, format_alphafold_data, annotate_accessibility, get_smooth_score, annotate_proteins_with_idr_pattern, get_extended_flexible_pattern, get_proximity_pvals, perform_enrichment_analysis, perform_enrichment_analysis_per_protein, evaluate_ptm_colocalization, extract_motifs_in_proteome
from structuremap.plotting import plot_enrichment, plot_ptm_colocalization


In [2]:
# Import 
import pandas as pd
import numpy as np
import os
import re
import plotly.express as px
import tqdm
import tempfile

In [3]:
# Set output directory to tempdir
output_dir = tempfile.gettempdir()

## Select proteins to use for this tutorial

In [4]:
test_proteins = ['O43353','P24941','Q92918','P45984','P28482','O96017',
                 'P02730','Q8NB16','Q13546','P29320','P08559','P15121']

## Download AlphaFold data

In [5]:
cif_dir = os.path.join(output_dir, 'tutorial_cif')
pae_dir = os.path.join(output_dir, 'tutorial_pae')

In [6]:
valid_proteins_cif, invalid_proteins_cif, existing_proteins_cif = download_alphafold_cif(
    proteins=test_proteins,
    out_folder=cif_dir)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 7701.86it/s]

2022-02-09 14:07:00> Valid proteins: 0
2022-02-09 14:07:00> Invalid proteins: 0
2022-02-09 14:07:00> Existing proteins: 12


In [7]:
valid_proteins_pae, invalid_proteins_pae, existing_proteins_pae = download_alphafold_pae(
    proteins=test_proteins,
    out_folder=pae_dir, 
    )

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 7414.80it/s]

2022-02-09 14:07:00> Valid proteins: 0
2022-02-09 14:07:00> Invalid proteins: 0
2022-02-09 14:07:00> Existing proteins: 12


## Format AlphaFold data input

In [8]:
alphafold_annotation = format_alphafold_data(
    directory=cif_dir, 
    protein_ids=test_proteins)

100%|████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  4.54it/s]


In [9]:
alphafold_annotation[0:3]

,protein_id,protein_number,AA,position,quality,x_coord_c,x_coord_ca,x_coord_cb,x_coord_n,y_coord_c,...,z_coord_ca,z_coord_cb,z_coord_n,secondary_structure,structure_group,BEND,HELX,STRN,TURN,unstructured
0,O43353,1,M,1,36.95,-20.649,-22.109,-22.991,-22.332,-11.261,...,-31.320,-30.655,-30.726,unstructured,unstructured,0,0,0,0,1
1,O43353,1,N,2,39.38,-18.674,-18.530,-17.736,-19.886,-8.995,...,-32.003,-33.314,-32.160,unstructured,unstructured,0,0,0,0,1
2,O43353,1,G,3,46.56,-17.165,-18.521,NaN,-18.591,-6.766,...,-29.807,NaN,-30.291,unstructured,unstructured,0,0,0,0,1


## Annotate pPSE values

In [10]:
full_sphere_exposure = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=24, 
    max_angle=180, 
    error_dir=pae_dir)

100%|████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:03<00:00,  3.83it/s]


In [11]:
alphafold_accessibility = alphafold_annotation.merge(
    full_sphere_exposure, how='left', on=['protein_id','AA','position'])

In [12]:
part_sphere_exposure = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=12, 
    max_angle=70, 
    error_dir=pae_dir)

100%|████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 65.67it/s]


In [13]:
alphafold_accessibility = alphafold_accessibility.merge(
    part_sphere_exposure, how='left', on=['protein_id','AA','position'])

In [14]:
alphafold_accessibility['high_acc_5'] = np.where(alphafold_accessibility.nAA_12_70_pae <= 5, 1, 0)
alphafold_accessibility['low_acc_5'] = np.where(alphafold_accessibility.nAA_12_70_pae > 5, 1, 0)

## Annotate IDRs

In [15]:
alphafold_accessibility_smooth = get_smooth_score(
    alphafold_accessibility, 
    np.array(['nAA_24_180_pae']), 
    [10])

100%|████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 36.08it/s]


In [16]:
alphafold_accessibility_smooth['IDR'] = np.where(
    alphafold_accessibility_smooth['nAA_24_180_pae_smooth10']<=34.27, 1, 0)

## Annottate short IDRs

In [17]:
alphafold_accessibility_smooth_pattern = annotate_proteins_with_idr_pattern(
    alphafold_accessibility_smooth,
    min_structured_length = 80, 
    max_unstructured_length = 20)

100%|███████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 507.14it/s]


In [18]:
alphafold_accessibility_smooth_pattern_ext = get_extended_flexible_pattern(
    alphafold_accessibility_smooth_pattern, 
    ['flexible_pattern'], [5])

100%|████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 26.94it/s]


In [19]:
alphafold_accessibility_smooth_pattern_ext[0:3]

,protein_id,protein_number,AA,position,quality,x_coord_c,x_coord_ca,x_coord_cb,x_coord_n,y_coord_c,...,TURN,unstructured,nAA_24_180_pae,nAA_12_70_pae,high_acc_5,low_acc_5,nAA_24_180_pae_smooth10,IDR,flexible_pattern,flexible_pattern_extended_5
0,O43353,1,M,1,36.95,-20.649,-22.109,-22.991,-22.332,-11.261,...,0,1,4,0,1,0,29.909091,1,0,0
1,O43353,1,N,2,39.38,-18.674,-18.530,-17.736,-19.886,-8.995,...,0,1,5,0,1,0,34.500000,0,0,0
2,O43353,1,G,3,46.56,-17.165,-18.521,NaN,-18.591,-6.766,...,0,1,6,0,1,0,38.461538,0,0,0


In [20]:
alphafold_accessibility_smooth_pattern_ext.columns


Index(['protein_id', 'protein_number', 'AA', 'position', 'quality',
       'x_coord_c', 'x_coord_ca', 'x_coord_cb', 'x_coord_n', 'y_coord_c',
       'y_coord_ca', 'y_coord_cb', 'y_coord_n', 'z_coord_c', 'z_coord_ca',
       'z_coord_cb', 'z_coord_n', 'secondary_structure', 'structure_group',
       'BEND', 'HELX', 'STRN', 'TURN', 'unstructured', 'nAA_24_180_pae',
       'nAA_12_70_pae', 'high_acc_5', 'low_acc_5', 'nAA_24_180_pae_smooth10',
       'IDR', 'flexible_pattern', 'flexible_pattern_extended_5'],
      dtype='object')

## Annotate PTM data

In [21]:
ptm_file_location = os.path.join("..","data","test_files","ptm_file.csv")
ptm_file = pd.read_csv(ptm_file_location)

In [22]:
ptm_file[0:3]

,Unnamed: 0,protein_id,AA,position,ac,ac_reg,ga,gl,gl_reg,m,m_reg,p,p_reg,sm,sm_reg,ub,ub_reg
0,0,O43353,K,17,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,O43353,K,182,0,0,0,0,0,0,0,0,0,0,0,1,0
2,2,O43353,K,203,0,0,0,0,0,0,0,0,0,0,0,1,0


In [23]:
alphafold_ptms = alphafold_accessibility_smooth_pattern_ext.merge(ptm_file, how='left', on=['protein_id','AA','position'])
alphafold_ptms = alphafold_ptms.fillna(0)

In [24]:
ptm_site_dict = {'p':['S','T','Y'],
                 'p_reg':['S','T','Y']}

## Perform PTM enrichment analysis

In [25]:
enrichment_p = perform_enrichment_analysis(
    df=alphafold_ptms, 
    ptm_types=['p', 'p_reg'], 
    rois=['IDR'], 
    ptm_site_dict = ptm_site_dict,
    quality_cutoffs=[0])

In [26]:
enrichment_p

,quality_cutoff,ptm,roi,n_aa_ptm,n_aa_roi,n_ptm_in_roi,n_ptm_not_in_roi,n_naked_in_roi,n_naked_not_in_roi,oddsr,p,p_adj_bf,p_adj_bh
0,0,p,IDR,281,259,116,165,143,639,3.141513,1.209934e-13,2.419868e-13,2.419868e-13
0,0,p_reg,IDR,60,259,21,39,238,765,1.730769,6.193364e-02,1.238673e-01,6.193364e-02


## Plot enrichment results

In [ ]:
plot_enrichment(data=enrichment_p,
                ptm_select=['p',
                            'p_reg'],
                roi_select=['IDR']
                )

## Perform PTM enrichment analysis per protein

In [ ]:
enrichment_p_per_protein = perform_enrichment_analysis_per_protein( 
    df=alphafold_ptms, 
    ptm_types=['p', 'p_reg'], 
    rois=['IDR'], 
    ptm_site_dict = ptm_site_dict,
    quality_cutoffs=[0])

In [ ]:
enrichment_p_per_protein

## Extract kinase motifs

In [ ]:
kinase_motifs_location = os.path.join("..","data","test_files","kinase_motifs.csv")
kinase_motifs = pd.read_csv(kinase_motifs_location, sep='\t')
kinase_motifs[0:3]

In [ ]:
kinase_motif_res = extract_motifs_in_proteome(
    alphafold_df=alphafold_ptms, 
    motif_df=kinase_motifs)
kinase_motif_res[0:3]

In [ ]:
# Format observed kinase motifs for merging with alphafold_ptms
kinase_motif_res['kinase_motif'] = 1
kinase_motif_res_sub = kinase_motif_res[['protein_id','position','AA','kinase_motif']].drop_duplicates()
alphafold_motifs = alphafold_ptms.merge(kinase_motif_res_sub, how='left', on=['protein_id','position','AA'])
alphafold_motifs = alphafold_motifs.fillna(0)
# alphafold_motifs[0:3]

In [ ]:
# test if phosphosites are enriched in kinase motifs
enrichment_p_inMotif = perform_enrichment_analysis(
    df=alphafold_motifs, 
    ptm_types=['p', 'p_reg'], 
    rois=['kinase_motif'], 
    ptm_site_dict=ptm_site_dict,
    quality_cutoffs=[0])

# inspect
enrichment_p_inMotif

In [ ]:
plot_enrichment(data=enrichment_p_inMotif,
                ptm_select=['p',
                            'p_reg'],
                roi_select=['kinase_motif']
                )

## Evaluate PTM colocalization

In [ ]:
p_colocalization = evaluate_ptm_colocalization(
    df=alphafold_ptms, 
    ptm_target='self',
    ptm_types=['p','p_reg'], 
    ptm_dict=ptm_site_dict,
    pae_dir=pae_dir,
    min_dist = 0.99,
    max_dist = 35,
    dist_step = 5)

In [ ]:
plot_ptm_colocalization(p_colocalization, context="3D")

## Get 3D clusters

In [ ]:
proximity_res_p = get_proximity_pvals(
    df=alphafold_ptms, 
    ptm_types = ['p'], 
    ptm_site_dict = ptm_site_dict, 
    error_dir=pae_dir, 
    per_site_metric= 'mean',
    error_operation='plus',
    n_random=10000, 
    random_seed=44)

In [ ]:
proximity_res_p